
# **Gestión de datos y datos digitales**
### **Autor: Ferran Carrascosa Mallafrè**

---
---

<!-- script html for image -->

<center>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c9/Star_Wars%2C_%C3%A9pisode_III_-_La_Revanche_des_Sith_logo.jpg/220px-Star_Wars%2C_%C3%A9pisode_III_-_La_Revanche_des_Sith_logo.jpg"  width="500" height="150"/>


<br>

Fuente de la imagen: [https://es.wikipedia.org](https://es.wikipedia.org/wiki/Star_Wars:_Episodio_III_-_La_venganza_de_los_Sith)

</center>

<br>

# **Índice**

---
---

> [Gestión de datos y datos digitales](#scrollTo=xIfYEg6Ud051)
<br>
>> 
>> [1.5.1. Introducción a las expresiones regulares](#scrollTo=zAhG3gpZgfxz&uniqifier=1) \\
>>
>> [1.5.2. Web Scraping](#scrollTo=Oi5_wNaf4rS7&uniqifier=1) \\
>>
>> [1.5.3. El inspector de código](#scrollTo=_1XsQgGD4rTO&uniqifier=1) \\
>>
>> [1.5.4. Beatiful soup](#scrollTo=Fexw2BNe4rTS&uniqifier=1)
>>
>> [1.5.5. Scraping contenido dinámico con Selenium](#scrollTo=EpT48am54rTa&uniqifier=1)



## 1.5. Web scraping. Uso de bots.

<br>

> Importante: El apartado de 1.5.5. Scraping contenido dinámico con Selenium, debe ejecutarse en un entorno local de Jupyetr notebook.


1.5.1. Introducción a las expresiones regulares

<br>

Las expresiones regulares permiten definir patrones de búsqueda por lo que suponen una alternativa, y a la vez, un soporte al web scraping.

Pueden ser tan simples como una cadena de texto como la del siguiente ejemplo. No obstante, antes de definir la expresión regular, primero se debe cargar la librería `re`.

<br>

In [ ]:
import re

ej1 = "Est3o xyz1234 esxyz5 un tXyzexto dexy6 ejexyzm7plo"

<br>

Ahora, se declara el patrón de la expresión regular.

<br>

In [ ]:
patron1 = re.compile('xyz')
print(patron1)

<br>

Para capturar todas las repeticiones en un lista.

<br>

In [ ]:
print(re.findall(patron1,ej1))

<br>

En vez de usar re.comple(), se puede utilizar directamente r"expresion_regular".

<br>

In [ ]:
print(re.findall(r"xyz",ej1))

<br>

Otra forma más flexible.

<br>

In [ ]:
for m in re.finditer(patron1, ej1):
    print('%02d-%02d: %s' % (m.start(), m.end(), m.group(0)))

<br>

Existen caracteres especiales: 

<br>

- .: Cualquier carácter.
- \d:  Cualquier dígito.
- \w: Cualquier carácter alfanumérico.
- \s: Cualquier espacio blanco como espacio nueva línea, retorno de carro, tabulador...

<br>

In [ ]:
patron2 = re.compile('\s\w\w\w\d..')
print(re.findall(patron2,ej1))

<br>

En este contexto, las mayúsculas indican negación:

<br>

- \D: Todo excepto un dígito.
- \W: Cualquier cosa excepto un carácter alfanumérico.
- \S: Cualquier carácter no espacio.

<br>

In [ ]:
patron3 = re.compile('\W\D')
print(re.findall(patron3,ej1))

<br>

Repeticiones de caracteres mediante corchetes, seguido del número de repeticiones.

<br>

In [ ]:
patron4 = re.compile('\d{2}')
print(re.findall(patron4,ej1))

<br>

Para repeticiones indefinidas de un carácter, se coloca inmediatamente después del carácter, uno de los siguientes símbolos:

<br>

- *: 0 o más repeticiones
- +: 1 o más veces

<br>

In [ ]:
patron5 = re.compile('\d*')
print(re.findall(patron5,ej1))

<br>
Ahora con el símbolo +.

<br>

In [ ]:
patron6 = re.compile('\d+')
print(re.findall(patron6,ej1))

<br>

Para decir que un carácter es opcional, se coloca inmediatamente después el símbolo ?.

<br>

In [ ]:
patron7 = re.compile('xyz?')
print(re.findall(patron7,ej1))

<br>

Si hay más de un carácter posible, se ponen éstos dentro de corchetes.

<br>

In [ ]:
patron8 = re.compile('[xX]yz')
print(re.findall(patron8,ej1))

<br>

Para decir que no tenemos ninguno de un conjunto posible de caracteres, se sitúan éstos, dentro de corchetes precedidos por el símbolo ^. Por ejemplo, [^xyz] indica que no sea x, y o z.

<br>

In [ ]:
patron9 = re.compile('[^xyz]\d')
print(re.findall(patron9,ej1))

<br>

Si indican rangos con -, por ejemplo [A-Z] o [0-9].

<br>

In [ ]:
patron9 = re.compile('[a-z]\d')
print(re.findall(patron9,ej1))

<br>

Para capturar un grupo de caracteres dentro de la expresión regular y desestimar el resto de caracteres, se utiliza paréntesis.

<br>

In [ ]:
for m in re.finditer(r"xyz(\d+)", ej1):
    print('%02d-%02d: %s' % (m.start(), m.end(), m.group(1)))

<br>

Las expresiones regulares son una potente herramienta para capturar contenido web. No obstante, no son la forma más robusta, ya que el contenido web, con mucha frecuencia, sufre de errores e inconsistencias en su construcción.

En los siguientes apartados se introducirá otros métodos que facilitan estas tareas.  

<br>

## 1.5.2. Web Scraping

<br>

Cuando no hay un API, se pueden utilizar las técnicas de web scraping para capturar su contenido.

El escenario más simple es cuando el contenido de la página está basado en HTML más estilo de texto CSS estático.

Por el contrario, cuando las páginas tienen contenido basado en HTML dinámico generado, por ejemplo, en JavaScript.

<br>

### HTML estático

<br>

Para comprender mejor la forma de obtener el contenido de una página html, se introducen a continuación, algunos conceptos básicos sobre HTML.

El html se basa en la estructura de tag:

<br>

<p style="text-align: center">&lt;nombre_tag *atributos*&gt; contenido &lt;/nombre_tag&gt;<p>

<br>

La estructura general de una página HTML es la siguiente:

<br>

- Se inicia la página con <!DOCTYPE html>. Así se especifica que el código es HTML5.
- El primer tag es &lt;html&gt; y el fichero finaliza con el correspondiente &lt;/html&gt;.
- La segunda sección es &lt;head&gt; y puede contener:
   - La sección &lt;title&gt; con el título de la página.
   - Referencias de estilo CSS con el tag &lt;link&gt; para dar formato al contenido de la página.
   - Enlaces a ficheros javascript con el tag: &lt;script&gt; utilizado para que el contenido sea dinámico.
- A continuación la sección &lt;body&gt; que puede contener:
   - Encabezados con &lt;h#&gt; (donde # es un número natural)
   - Texto con &lt;p&gt;.
   - Hipervínculos con el atributo **href** del tag &lt;a&gt;. p.e.: &lt;a hrf = "www.ine.es" &gt; INE &lt;/a&gt;.  
   - Imágenes con el atributo **src** del tag &lt;img&gt;. p.e.: &lt;img src = "my_pic.jpg" /&gt;.

Puedes ampliar la información en la página del consorcio <a href="http://www.w3.org/community/webed/wiki/HTML">World Wide Web</a>.

Veamos un ejemplo:

<br>

```
<!DOCTYPE html>
<html>
<head>
<title>Hello World!</title>
</head>
<body>
<h1>Titulo 1</h1>
<p>Texto 1</p>
<h2>Titulo 2</h2>
<p>Texto con enlace a <a href = "https://en.wikipedia.org/wiki/Star_Wars" > Star Wars Wikipedia </a>.</p>
<img src="https://en.wikipedia.org/wiki/File:Star_wars2.svg">
</body>
</html>
```
<br>

In [ ]:
from IPython.core.display import display, HTML
display(HTML('''
<!DOCTYPE html>
<html>
<head>
<title>Hello World!</title>
</head>
<body>
<h1>Titulo 1</h1>
<p>Texto 1</p>
<h2>Titulo 2</h2>
<p>Texto con enlace a <a href = "https://es.wikipedia.org/wiki/Star_Wars" > Star Wars Wikipedia </a>.</p>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/6c/Star_Wars_Logo.svg/250px-Star_Wars_Logo.svg.png">
</body>
</html>
'''))

<br>

### Listas HTML

<br>

La forma clásica de dar formato al contenido en html estático es mediante listas y tablas. 

Una lista puede ser:

<br>

- Numerada u ordenada: Se definen con &lt;ol&gt;.
- No ordenada: Se definen con &lt;ul&gt;.

<br>

Sus elementos se insertan mediante el tag &lt;li&gt;.

Veamos un ejemplo:

<br>

```
<ol>
  <li>Café</li>
  <li>Té</li>
  <li>Leche</li>
</ol>
```

<br>

Resultado:

<br>

<ol>
  <li>Café</li>
  <li>Té</li>
  <li>Leche</li>
</ol>

<br>

### Tablas HTML

Para definir tablas, se utiliza el tag &lt;table&gt;. Cada  fila se define con &lt;tr&gt; y las columnas a través de sus elementos con el tag &lt;td&gt;.

Pueden contener un encabezado &lt;thead&gt; y un cuerpo &lt;tbody&gt;.

Cada elemento del encabezado se define con &lt;th&gt;.

Si la celda tiene varias columnas se utiliza el atributo *colspan=número de celdas*. 

Veamos un ejemplo:

<br>

```
<table>
<thead>
<tr><th colspan = 2>Una tabla</th><tr>
</thead>
<tbody>
<tr>
<td>Elmento 1</td><td>Elmento 2</td>
</tr>
<tr>
<td colspan=2>Elmento 3</td>
</tr>
</tbody>
</table>
```

<br>

<table>
<thead>
<tr><th colspan = 2>Una tabla</th><tr>
</thead>
<tbody>
<tr>
<td>Elmento 1</td><td>Elmento 2</td>
</tr>
<tr>
<td colspan=2>Elmento 3</td>
</tr>
</tbody>
</table>

<br>

### Tablas HTML con estilos 

<br>

Actualmente se combina el html con estilos mediante &lt;div&gt; y &lt;span&gt;:

<br>

- div: Permite definir bloques (o celdas) con un estilo.
- span: Permite diferenciar un fragmento del texto con un estilo. 

<br>

Veamos un ejemplo:

<br>

```
<div style = "background-color:yellow;padding:10px;display:inline-block">  Text 1
</div>
<div style = "background-color:red;padding:10px;display:inline-block">  Text 2
</div>
<div style = "background-color:green;padding:10px;display:inline-block">  Text 3
</div>
```

<br>

El resultado:

<br>

<div style = "background-color:yellow;padding:10px;display:inline-block">  Text 1
</div>
<div style = "background-color:red;padding:10px;display:inline-block">  Text 2
</div>
<div style = "background-color:green;padding:10px;display:inline-block">  Text 3
</div>

<br>

### Estilos CSS

<br>

Los ficheros CSS permiten definir un conjunto de reglas de estilo que se podrán aplicar al contenido html. 

<br>

#### Selector CSS

<br>

El formato general es:

<br>

```
selector_css { propiedad: valor; } 
```

<br>

El *selector_css* puede ser cualquier tag html. De esta forma se puede modificar el estilo de un conjunto de elementos. 

Para modificar el contenido de los tags div.

<br>

```
div { color:green; }
```

<br>

#### Clases

<br>

Para agrupar un conjunto de tags bajo un mismo estilo, se utiliza el atributo **class** de los tags &lt;div&gt; o &lt;p&gt;.

<br>

```
< div class = "mi_clase" >  o también < p class = "mi_clase">
```

<br>

Para modificar el estilo en el fichero CSS se utiliza el nombre de la clase precedido por punto.

<br>

```
.mi_clase { font-family:Arial; } 
```

<br>

#### Identificadores

<br>

Para marcar un elemento y aplicarle un cierto estilo, se pueden utilizar los identificadores. Para esto se utiliza el atributo ID.

<br>

```
< div id = "mi_ID" >
```

<br>

Para modificar el estilo en ficheros de tipo CSS, se utiliza el nombre precedido de la almohadilla.

<br>

```
#mi_ID { font-size:16px; }
```

<br>

### Selección CSS

<br>

Los tags, así como sus atributos, constituyen una estructura arbórea, donde el tag html es el nodo raíz. 

Para facilitar la selección de nodos de este árbol, CSS define un conjunto de reglas de selección. 

Dos ejemplos de reglas habituales de selección son.

<br>

+ "elem1 elem2": Se refiere a cualquier elem2 dentro de cualquier otro elem1, sin tener en cuenta el grado de anidación.
+ "elem1>elem2": Se refiere a los elem2 hijos directos de elem1.

<br>

Para seleccionar los elementos identificados por un atributo concreto, se utiliza:

<br>

```
elemento[attribute = value]
```

<br>

Por ejemplo para seleccionar botones.

<br>

```
input[type = "button"]
```

<br>

Otros patrones de búsqueda: 

<br>

- atributo~=valor: Contienen la palabra completa en una lista de palabras separadas por espacio.
- atributo|=valor: Contienen en una lista separada por guiones.
- atributo^=valor: Contienen, al inicio, el valor.
- atributo$=valor: Contienen, al final, el valor.
- atributo\*=valor: Contienen el valor como parte de alguna palabra.

<br>

Se pueden encadenar varios selectores:

<br>

```
elemento[atributo1 = valor1][atributo2 = valor2]
```

<br>

Para ampliar las reglas de selección CSS puedes visitar la [referencia de selectores CSS](https://www.w3schools.com/cssref/css_selectors.asp).

A continuación, se presenta un ejemplo de selección de texto de la página Star Wars de la Wikipedia.

En primer lugar se obtiene el texto html y se le da una estructura de árbol.

<br>

In [ ]:
from urllib.request import urlopen

source = urlopen('https://es.wikipedia.org/wiki/Star_Wars')

from lxml import html
from lxml import cssselect
tree = html.document_fromstring(source.read())

<br>

A continuación, se capturan todas las citas de autores mediante el tag blockquote, agrupados bajo el estilo flexquote. El selector CSS correspondiente es: *blockquote.flexquote*.

<br>

In [ ]:
quotes = tree.cssselect("blockquote.flexquote")

len(quotes)

<br>

Observamos que se han capturado 5 citas. Veamos la primera. 

<br>

In [ ]:
print(quotes[0].text_content())

<br>

Para capturar todos los enlaces (tag a) donde el enlace (href^=) empieza por *http*.

In [ ]:
links = tree.cssselect('a[href^="http"]')
len(links)

<br>

Muchos enlaces! Veamos los 5 primeros.

<br>

In [ ]:
linksList = [x.values()[0] for x in links]
linksList[0:5]

<br>

Más difícil aún, veamos como capturar una tabla de datos. En este caso, capturamos la tabla que cumple con la regla CSS siguiente: *table.wikitable:nth-child(135) > tbody:nth-child(1)*

El primer paso consiste en capturar la tabla.

<br>

In [ ]:
tables = tree.cssselect("table.wikitable:nth-child(135) > tbody:nth-child(1)")

<br>

El segundo paso, consiste en capturar cada una de las filas (tag tr).

<br>

In [ ]:
table_rows = tables[0].findall("tr")

<br>

Por último, hay que parsear cada fila para separar el contenido de las columnas (tag td).

<br>

In [ ]:
tablFinal = []

for tr in table_rows:
    td = tr.findall("td")
    row = [i.text for i in td]
    tablFinal.append(row)

print(tablFinal)

<br>

Bien! Misión cumplida Padawan!

Revisa el código anterior, y verás que todo el truco consiste en saber el selector CSS, éste, a veces se expresa en forma de reglas fáciles de deducir, como *a[href^="http"]*. Otras veces, se expresa en formas tan complejas como esta *table.wikitable:nth-child(135) > tbody:nth-child(1)*.

Para saber cómo se obtienen estas expresiones, lee el siguiente apartado sobre el inspector de código.

<br>

## 1.5.3. El inspector de código

<br>

Como ya se ha anticipado, una pieza fundamental para recuperar el contenido deseado de una página web y darle estructura, son las distintas reglas para seleccionar elementos del árbol de nodos.

Para facilitar este análisis, se pueden utilizar tanto Chrome como Firefox, las herramientas de apoyo al desarrollador que verás a continuación.

<br>

### Código fuente HTML

<br>

Para acceder al código fuente de una página que tenemos abierta en el navegador, es tan simple como apretar el botón derecho en cualquier ubicación de la página (que no sea una imagen o un enlace) y apretar la opción de "Ver código fuente de la página" (en Firefox) o  "Visualizar origen de la página" (en Chrome).

De esta forma se puede revisar detalles del código HTML estático. No obstante, esta forma no es la más ágil para analizar aspectos concretos de la web, ya que el código, a menudo es muy voluminoso y difícil de comprender.

<br>

### Inspeccionar elementos

<br>

Cuando estamos interesados en conocer la forma de identificar de forma unívoca un contenido de la web, la forma más eficaz, es el inspector de elementos. 

Para basarnos en un ejemplo común, ve al enlace de la [primera cita](https://es.wikipedia.org/wiki/Star_Wars#Trilog%C3%ADa_original) de George Lucas en la wikipedia.

<br>

![](www/cita1.png)

\<imagen1\>Imagen: Cita de George Lucas (Fuente: [Wikipedia](https://es.wikipedia.org/wiki/Star_Wars#Trilog%C3%ADa_original) ). (www/cita1.png){width=850px}

<br>

A continuación, clica botón derecho sobre el texto de la cita y aprieta la opción "Inspeccionar elemento" en Firefox, o bien,  "Inspecciona" en Chrome. Verás que automáticamente se abre una consola dentro de la página en la parte inferior (o derecha) de la página. 

El inspector, abre la puerta a un sin fin de opciones de análisis, útiles para el web scraping. Por ejemplo, observa como al mover el puntero sobre las líneas del inspector de código (parte inferior, o derecha), automáticamente se ilumina el contorno del elemento web al que hace referencia. 

El siguiente paso es clicar (botón izquierdo) sobre la línea con texto `<blockquote class="flexquote">`. En Firefox, deberías ver lo siguiente (En Chrome debería ser parecido).

<br>

![](www/cita2.png)

\<imagen1\>Imagen: Selector CSS de la cita de George Lucas (Fuente: [Wikipedia](https://es.wikipedia.org/wiki/Star_Wars#Trilog%C3%ADa_original) ). (www/cita2.png){width=950px}

<br>

Finalmente, sin mover el cursor de la línea, aprieta el botón derecho y mueve el cursor sobre el menú contextual "Copiar" > y aprieta la opción Selector CSS.

<br>

![](www/cita3.png)

\<imagen1\>Imagen: Selector CSS de la cita de George Lucas (Fuente: [Wikipedia](https://es.wikipedia.org/wiki/Star_Wars#Trilog%C3%ADa_original) ). (www/cita3.png){width=350px}

<br>

Si todo ha ido bien, el texto obtenido al copiar, debería ser: "blockquote.flexquote:nth-child(56)". Compruébalo pegando el texto en cualquier editor de texto.

Observa también, que si se elimina la última parte ":nth-child(56)", nos quedamos con "blockquote.flexquote", que es el selector CSS deseado de "Todas las citas".

Practica con otros elementos, por ejemplo, con la tabla de la sección de [Crítica](https://es.wikipedia.org/wiki/Star_Wars#Cr%C3%ADtica) que se ha descargado en el apartado anterior. ¿Obtienes el mismo selector? 
<br>

### Selección con XPATH

El lenguaje XPATH  es una alternativa basada en XML que permite parsear el contenido de una página HTML.

Veamos algunos ejemplos de su sintaxis.

<br>

- // : Indica en cualquier lugar de la página. 
- /  : Indica que es descendiente directo
- @ : Para referirse a cierto atributo. 
- . : Nodo actual. 
- .. : Nodo padre. 
- \* : Indica cualquier elemento nodo. 
- @* : Cualquier atributo.

<br>

Por ejemplo,

<br>

- bookstore/book: Todos los book que son hijos directos de bookstore.
- bookstore//book: Todos los book que descienden de bookstore, no importa en qué lugar.
- //title[@*]: Selecciona todos los elementos title que tienen al menos un atributo de cualquier tipo.
- /bookstore/book[1]: El primer book hijo de bookstore
- /bookstore/book[last()]: El último book hijo de bookstore
- /bookstore/book[price>35.00]: Los libros con precio superior a 35. 

<br>

Ver más ejemplos en [W3 Schools](https://www.w3schools.com/xml/xpath_syntax.asp).

<br>

#### Inspector de código XPATH

<br>

De la misma forma que el selector CSS, tanto Chrome como Firefox permiten obtener los selectores XPATH. 

Veamos el mismo ejemplo de la [cita de George Lucas](https://es.wikipedia.org/wiki/Star_Wars#Trilog%C3%ADa_original). Abre de nuevo la página e Inspecciona la cita con el botón derecho > Inspecciona elemento.  Sitúate sobre la línea adecuada y clica botón derecho > Copiar > Xpath.

Deberías obtener el texto: /html/body/div[3]/div[3]/div[5]/div[1]/blockquote[1]

¿Correcto?

En este caso, código para obtener "todas las citas" en XPATH podría quedar como sigue. 

<br>

In [ ]:
citasX = tree.xpath('//blockquote')
len(citasX)

<br>

Vemos que efectivamente se han obtenido 5 citas como en el selector CSS. Veamos la primera.

<br>

In [ ]:
print(citasX[0].text_content())

<br>

Conseguido! Probemos ahora con el selector Xpath de la tabla de la sección de [Crítica](https://es.wikipedia.org/wiki/Star_Wars#Cr%C3%ADtica).

¿Coincide con el siguiente?

<br>

- */html/body/div[3]/div[3]/div[5]/div[1]/table[7]/tbody*

<br>

## 1.5.4. Beatiful soup

<br>

Esta librería de Python busca apoyar el sistema de navegación definido por los árboles de selectores CSS y XPATH.  

Preparar la sopa maravillosa es tan simple como lo siguiente. 

<br>

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import lxml

source = urlopen('https://es.wikipedia.org/wiki/Star_Wars')
soup = BeautifulSoup(source, 'lxml')


<br>

Ver el título de la página.

<br>

In [ ]:
soup.title.string

<br>

Obtener los enlaces (tag a).

<br>

Texto de la página

<br>

In [ ]:
print(soup.get_text()[0:200])

<br>

Acceder a "todas las tablas".

<br>

In [ ]:
tables = soup.select("table.wikitable > tbody")

<br>

Ojo, ahora hay que procesarlas igual que antes.

<br>

### Parseado de tablas con Pandas

<br>

La librería Pandas, por su lado, ofrece una solución muy rápida y efectiva para obtener las tablas HTML.

Si estas tablas están construidas con código HTML estático, obtenerlas es tan simple como realizar lo siguiente.

<br>

In [ ]:
import pandas as pd
dfs = pd.read_html("https://es.wikipedia.org/wiki/Star_Wars")

<br>

Nuestra tabla de interés es la 6.

<br>

In [ ]:
dfs[6]

Fantástico! Pandas, como siempre, da la mejor solución cuando se trata de trabajar con tablas.

<br>

## 1.5.5. Scraping contenido dinámico con Selenium

Selenium es una herramienta creada originalmente para testear aplicaciones web a través del propio navegador controlado desde Python y Java. Permite clicar los botones, rellenar formularios...

Aunque no es recomendable, se puede utilizar Selenium desde Colab. Para ello, primero hay que instalar algunos paquetes.

In [ ]:
from IPython import get_ipython
if 'google.colab' in str(get_ipython()):
  # instalar drivers
  !apt install chromium-chromedriver
  !cp /usr/lib/chromium-browser/chromedriver /usr/bin
  !pip install selenium

Para iniciar Selenium en local, obviamente es necesario tener instalado un navegador. Admite los navegadores principales: Chrome, Firefox, Edge, IE,  Safari y Opera.

En este caso voy a utilizar Chrome pero se puede utilizar cualquier otro.

También para trabajar en local, es necesario tener el webdriver correspondiente descargado y guardado en una ruta visible (p.e. dentro del PATH). Los drivers se encuentran a partir de la [página de Selenium](https://www.selenium.dev/documentation/webdriver/getting_started/install_drivers/).

En el caso de Chrome puedes ir directamente al [Crome Driver](https://sites.google.com/chromium.org/driver/).

Una vez descargado el driver conveniente, hay que guardarlo (una vez hayas descomprimido el fichero) en una ruta accesible. La recomendación es utilizar la ruta local donde esté guardo el notebook. 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
if 'google.colab' in str(get_ipython()):
  # instalar drivers
  options = webdriver.ChromeOptions()
  options.add_argument('-headless')
  options.add_argument('-no-sandbox')
  options.add_argument('-disable-dev-shm-usage')
  #options.headless = True
  #options.add_argument("--window-size=1920,1200")
  driver = webdriver.Chrome('chromedriver',options=options)
else:
  import os 
  os.environ["PATH"] = '$PATH:.'
  #driver = webdriver.Chrome('C:/Users/usuari/Documents/masterM2/Data_Digital/chromedriver')
  driver = webdriver.Firefox()


<br>

Si estás ejecutando en local, verás cómo se abre Chrome. 

A continuación, vemos cómo obtener los datos de Google. 

<br>

In [ ]:
driver.get("https://www.google.com/")
print(driver.page_source[:500])

<br>

Algunos métodos relevantes.

<br>

- .get(url): Abrir URL
- .find_element(By.METODO, "elmento") : Busca el elemento con uno de los siguientes métodos:
   - By.CLASS_NAME: Por nombre de la clase.
   - By.CSS_SELECTOR: Por selector CSS.
   - By.ID: Por ID.
   - By.LINK_TEXT: que contenga texto.
   - By.PARTIAL_LINK_TEXT: que contenga texto parcialmente.
   - By.NAME: Por nombre.
   - By.TAG_NAME: Por nombre del tag.
   - By.XPATH: Por XPATH.
- .page_source: Devuelve el código HTML. Observa que ahora ya está resuelta la parte de html dinámico (o javascript).

<br>

Otras utilidades:

<br>

- .execute_script('nombre.javascript()') : Ejecuta javascript.
- .save_screenshot('spam.png'): guarda la imagen.
- .switch_to_alert(): Gestiona los pop-upps.
- .forward() / .back(): Navegar...

<br>

Veamos un ejemplo de captura de productos de Star Wars devueltos por Google y que cumplen con las siguientes condiciones:

<br>

- Son productos nuevos
- Valen más de 1.000 Euros.

<br>

Se abre la página de Google.

<br>

In [ ]:
driver.get("https://www.google.com/")

<br>

Se elimina el Pop-up y se realiza la búsqueda en la tienda.

<br>

In [ ]:
try:
  driver.find_element(By.ID, "L2AGLb").click()
except:
  print("Ya esta inicializada")

In [ ]:
driver.find_element(By.NAME, "q").send_keys("star wars")

In [ ]:
driver.find_element(By.NAME, "q").send_keys(Keys.ENTER)

In [ ]:
driver.find_element(By.LINK_TEXT, "Shopping").click()

<br>

Se filtra por productos nuevos y que sean de valor superior a 1.000 euros.

<br>

In [ ]:
driver.find_element(By.CSS_SELECTOR, "div.sh-dr__g:nth-child(10) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > span:nth-child(1) > div:nth-child(1) > a:nth-child(1) > span:nth-child(1)").click()

In [ ]:
driver.find_element(By.NAME, "lower").click()

In [ ]:
driver.find_element(By.NAME, "lower").send_keys("1000")

In [ ]:
driver.find_element(By.NAME, "lower").send_keys(Keys.ENTER)

<br>

Se captura el texto en html.

<br>

In [ ]:
html_text = driver.page_source

<br>

Se convierte a estructura de árbol.

<br>

In [ ]:
from lxml import html
from lxml import cssselect
tree = html.document_fromstring(html_text)

<br>

Se seleccionan los productos y se extrae el contenido del texto.

<br>

In [ ]:
# se captura el nombre, el precio y la url
star_wars_prod1 = tree.cssselect("h4.Xjkr3b")
star_wars_prod2 = tree.cssselect("span.a8Pemb.OFFNJ")
star_wars_prod3 = tree.cssselect("a.shntl")

prod_list = []
for prod1,prod2,prod3 in zip(star_wars_prod1,star_wars_prod2,star_wars_prod3):
  auxTxt = {'nombre':prod1.text_content(), 
            'precio': prod2.text_content(), 
            'href': re.findall(r'(https?://\S+)',prod3.attrib['href'])[0]}
  prod_list.append(auxTxt)

df = pd.DataFrame(prod_list)

df.tail()

In [ ]:
df.shape

In [ ]:
driver.quit()

<br>

Como ya has visto, existe un método para hacer clic.

- **.click()** - click on a selected element</li>

<br>

Métodos para acceder a las propiedades:

<br>

- .location: posición x, y.
- .parent: nodo padre.
- .tag_name: el tag del elemento.
- .text: texto del elemento de sus hijos.

<br>

- Acciones encadenadas con ActionChains.

<br>

En cuanto a tiempos de espera, existen dos tipos de estrategias. La implícita y la explícita.

<br>

- .implicitly_wait(seconds): La implícita, especifica un tiempo de espera.
- La explícita, le dice al driver que espere hasta que se cumpla cierta condición (p.e. hasta que se cargue la página).

<br>

Veamos un ejemplo de código.

<br>

```
try:
  wait = webdriverwait(browser,10)
  
  element = wait.until(EC.element_to_be_clickable((By.ID,'someid')))
except:
  print 'Time out!!'
```

<br>

Mira [Selecium waits](https://selenium-python.readthedocs.io/waits.html) para más información.

<br>

#### Otros recursos de selenium

<br>

- [Selenium CheatSheet](http://www.cheat-sheets.org/saved-copy/rc067-010d-selenium-1.pdf)

<br>

## Resumen

<br>

- Las expresiones regulares permiten acceder al texto mediante patrones.
- Los selectores CSS y XPATH aseguran accesos estables a la información.
- Para superar las barreras del html dinámico se puede utilizar un dirver que controle el navegador.

<br>